In [3]:
# step1 : Install CatBoost 

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd 
import seaborn as sns 

In [4]:

# Step 2: Load Titanic dataset
df = sns.load_dataset('titanic')
df = df[['survived', 'pclass', 'sex', 'age', 'embarked']].dropna()

In [5]:
X = df.drop("survived",axis=1)
y=df["survived"]


In [6]:
cat_features  = ['sex',"embarked","pclass"]


In [7]:
X_train,y_train, X_test,y_test = train_test_split(X,y,test_size=0.2, random_state=42)


In [8]:
# Step 1: Install CatBoost (only once)
# !pip install catboost

from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import seaborn as sns

# Step 2: Load Titanic dataset
df = sns.load_dataset('titanic')
df = df[['survived', 'pclass', 'sex', 'age', 'embarked']].dropna()

# Step 3: Split data
X = df.drop('survived', axis=1)
y = df['survived']

# Identify categorical features
cat_features = ['sex', 'embarked', 'pclass']

# Step 4: Train/Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 5: Train CatBoost
model = CatBoostClassifier(verbose=0)
model.fit(X_train, y_train, cat_features=cat_features)

# Step 6: Evaluate
y_pred = model.predict(X_test)
print("Accuracy of test data :", accuracy_score(y_test, y_pred))


Accuracy of test data : 0.7902097902097902


In [9]:
#testing on train data to check overfitting 

y_train_pred= model.predict(X_train)
print("Accuracy of train data: ",accuracy_score(y_train,y_train_pred))

Accuracy of train data:  0.8295254833040422


### since we the accuracy is low we are using other steps to imporve our accuracy 

In [10]:
#add age_grp
def age_group(age):
    if age<18:
        return "child"
    elif age<30:
        return "young"
    elif age<50:
        return "adult"
    else:
        return "senior"
    
X_train["age_group"] = X_train['age'].apply(age_group)
X_test["age_group"] = X_test['age'].apply(age_group)


#add sex_pclass 

X_train['sex_pclass'] = X_train['sex'] + "_" +X_train['pclass'].astype(str)


X_test['sex_pclass'] = X_test['sex'] + "_" +X_test['pclass'].astype(str)

In [11]:
# training the catboost model using regularization

# Update cat_features to include all categorical columns in X_train/X_test
cat_features_updated = ['sex', 'embarked', 'pclass', 'age_group', 'sex_pclass']

model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    l2_leaf_reg=4,
    random_strength=1,
    od_type='Iter',
    od_wait=50,
    cat_features=cat_features_updated,
    verbose=100
)
model.fit(X_train, y_train, eval_set=(X_test, y_test))

0:	learn: 0.6721950	test: 0.6750795	best: 0.6750795 (0)	total: 13.4ms	remaining: 6.67s
100:	learn: 0.3535925	test: 0.4889033	best: 0.4855161 (74)	total: 1.7s	remaining: 6.7s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.4855161351
bestIteration = 74

Shrink model to first 75 iterations.


In [12]:
# Train accuracy
y_train_pred = model.predict(X_train)
print("Accuracy on train data:", accuracy_score(y_train, y_train_pred))

# Test accuracy
y_test_pred = model.predict(X_test)
print("Accuracy on test data:", accuracy_score(y_test, y_test_pred))


Accuracy on train data: 0.8242530755711776
Accuracy on test data: 0.7832167832167832


In [13]:
%pip install ipywidgets

## cross validation to stabilize accuracy 
from catboost import Pool, cv

#combine X, y into one pool
full_pool = Pool(X,y,cat_features = cat_features)

params = {
    'iterations': 1000,
    'learning_rate': 0.03,
    'depth': 6,
    'l2_leaf_reg': 4,
    'random_strength': 1,
    'loss_function': 'Logloss',
    'od_type': 'Iter',
    'od_wait': 50,
    'verbose': False
}

#run 5 fold cross validation
cv_result = cv(
    params = params,
    pool=full_pool,
    fold_count=5,
    shuffle=True,
    partition_random_seed=42,
    plot = True)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Training on fold [0/5]

bestTest = 0.4985629232
bestIteration = 73

Training on fold [1/5]

bestTest = 0.3903073129
bestIteration = 163

Training on fold [2/5]

bestTest = 0.3447323829
bestIteration = 271

Training on fold [3/5]

bestTest = 0.5214055632
bestIteration = 90

Training on fold [4/5]

bestTest = 0.343120209
bestIteration = 924



In [14]:
mean_iter = int((73 + 163 + 271 + 90 + 924) / 5)
print(mean_iter)  # ≈ 304


304


In [21]:
final_model = CatBoostClassifier(
    iterations=305,
    learning_rate=0.1,
    depth=5,
    l2_leaf_reg=5,
    random_strength=1,
    cat_features=cat_features_updated,
    verbose=100
)

final_model.fit(X_train, y_train, eval_set=(X_test, y_test))

# Final evaluation
from sklearn.metrics import accuracy_score

y_train_pred = final_model.predict(X_train)
y_test_pred = final_model.predict(X_test)

print("Final Train Accuracy:", accuracy_score(y_train, y_train_pred))
print("Final Test Accuracy:", accuracy_score(y_test, y_test_pred))


0:	learn: 0.6525154	test: 0.6582306	best: 0.6582306 (0)	total: 18.1ms	remaining: 5.5s
100:	learn: 0.3283837	test: 0.4883637	best: 0.4877328 (77)	total: 1.65s	remaining: 3.33s
200:	learn: 0.2689162	test: 0.4987983	best: 0.4877328 (77)	total: 3.31s	remaining: 1.71s
300:	learn: 0.2249494	test: 0.5063736	best: 0.4877328 (77)	total: 5s	remaining: 66.5ms
304:	learn: 0.2240405	test: 0.5048494	best: 0.4877328 (77)	total: 5.06s	remaining: 0us

bestTest = 0.4877328339
bestIteration = 77

Shrink model to first 78 iterations.
Final Train Accuracy: 0.8242530755711776
Final Test Accuracy: 0.7902097902097902
